In [7]:
# B1 - DELEGATION ROUTING ALGORITHM & UNIT TESTS #
import random, time
# ------------------------------------------------------------------
# add_delegation(): Adds src → dst unless it creates a cycle
# ------------------------------------------------------------------
def add_delegation(delegation, src, dst):
    cur = dst
    while cur is not None:
        if cur == src:
            raise ValueError(f"Cycle blocked: {src} → {dst}")
        cur = delegation.get(cur)
    delegation[src] = dst

# ------------------------------------------------------------------
# resolve_delegations(): Resolves chains to final voters with path compression
# ------------------------------------------------------------------
def resolve_delegations(delegation):
    final = {}
    for u in delegation:
        # walk from u until you hit a terminal or a memoized node
        path = []
        v = u
        while v not in final and delegation.get(v) is not None:
            path.append(v)
            v = delegation[v]
        # v is now either in final or a terminal (delegation[v] is None)
        root = final.get(v, v)
        # path–compress
        for x in path:
            final[x] = root
        # also remember u itself (in case path was empty)
        final[u] = root
    return final

# ------------------------------------------------------------------
# Verbose test-suite
# ------------------------------------------------------------------
# a) Long chain  A→B→C→D→E
chain = {'A':'B','B':'C','C':'D','D':'E','E':None}
print("Test (a):", resolve_delegations(chain))
assert all(resolve_delegations(chain)[u] == 'E' for u in chain)

# b) Three branches converging to a single sink
branch = {'X':'M','Y':'M','Z':'M','M':'N','N':'O','O':None}
print("Test (b): sink →", set(resolve_delegations(branch).values()).pop())
assert set(resolve_delegations(branch).values()) == {'O'}

# c) Attempt to close a loop: D→A should be rejected
safe = {'A':'B','B':'C','C':'D','D':None}
try:
    add_delegation(safe, 'D', 'A')
except ValueError as e:
    print("Test (c):")
    print(e)

# d) Mixed graph: add S→Q (valid), block W→U (cycle)
mixed = {'P':'Q','Q':'R','R':None}
add_delegation(mixed, 'S', 'Q')          # merges into R
mixed.update({'U':'V','V':'W'})
try:
    add_delegation(mixed, 'W', 'U')      # cycle blocked
except ValueError as e:
    print(e)
print("Test (d):", resolve_delegations(mixed))

# e) Large random acyclic graph
N = 1000000
users = [f'U{i}' for i in range(N)]
big   = {u: None for u in users}
# Build graph in O(N)
for i, src in enumerate(users):
    if i > 0 and random.random() < 0.5:
        dst = users[random.randrange(i)]
        add_delegation(big, src, dst)

t0 = time.time()
out_big = resolve_delegations(big)
print(f"Test (e): {N:,}-node graph resolved in {time.time()-t0:.3f}s")

print("\nAll delegation-routing tests passed.")

Test (a): {'A': 'E', 'B': 'E', 'C': 'E', 'D': 'E', 'E': 'E'}
Test (b): sink → O
Test (c):
Cycle blocked: D → A
Cycle blocked: W → U
Test (d): {'P': 'R', 'Q': 'R', 'R': 'R', 'S': 'R', 'U': 'W', 'V': 'W'}
Test (e): 1,000,000-node graph resolved in 1.209s

All delegation-routing tests passed.
